# Inicialización

In [ ]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
library(data.table)
library(ggplot2)
library(umap)
library(dbscan)
library(FNN)
library(knitr)
library(kableExtra)

In [ ]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset <- fread("~/datasets/competencia_02_crudo.csv.gz" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

In [ ]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

In [ ]:
# Salsa Magica para 202106
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

In [ ]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

# variable extraida de una tesis de maestria de Irlanda, se perdió el link
dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

Sys.time()

In [ ]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

## lags de orden 3
#dataset[,
#  paste0(cols_lagueables, "_lag3") := shift(.SD, 3, NA, "lag"),
#  by= numero_de_cliente,
#  .SDcols= cols_lagueables
#]
#
## lags de orden 6
#dataset[,
#  paste0(cols_lagueables, "_lag6") := shift(.SD, 6, NA, "lag"),
#  by= numero_de_cliente,
#  .SDcols= cols_lagueables
#]
#
## lags de orden 12
#dataset[,
#  paste0(cols_lagueables, "_lag12") := shift(.SD, 12, NA, "lag"),
#  by= numero_de_cliente,
#  .SDcols= cols_lagueables
#]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
  #dataset[, paste0(vcol, "_delta3") := get(vcol) - get(paste0(vcol, "_lag3"))]
  #dataset[, paste0(vcol, "_delta6") := get(vcol) - get(paste0(vcol, "_lag6"))]
  #dataset[, paste0(vcol, "_delta12") := get(vcol) - get(paste0(vcol, "_lag12"))]
}

Sys.time()

In [ ]:
# --- Crear la clase para clustering ---
dataset[, clase_cluster := ifelse(
  clase_ternaria %in% c("BAJA+1", "BAJA+2"), "BAJA",
  ifelse(clase_ternaria == "CONTINUA", "CONTINUA", NA)
)]

In [ ]:
# --- Reconstruir antiguedad en meses ---
dataset[, antiguedad_mes :=
      (foto_mes %/% 100 - min(foto_mes %/% 100, na.rm=T))*12 +
      (foto_mes %% 100 - min(foto_mes %% 100, na.rm=T)),
   by = numero_de_cliente]

In [ ]:
# --- Filtrar ---
df_filtrado <- dataset[antiguedad_mes >= 12 | clase_cluster == "BAJA"]

In [ ]:
training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104
)

df_train <- df_filtrado[foto_mes %in% training]

df_202106 <- df_filtrado[foto_mes == 202106]

cat("Training rows:", nrow(df_train), "\n")
cat("202106 rows:", nrow(df_202106), "\n")

Seleccionamos variables monetarias

In [ ]:
vars_monetarias <- names(df_filtrado)[grepl("^(m|Master_m|Visa_m)", names(df_filtrado))]

df_m <- df_filtrado[, ..vars_monetarias]
df_m[is.na(df_m)] <- 0

Estandarizamos

In [ ]:
df_scaled <- scale(df_m)

PCA

res_pca <- prcomp(df_scaled, scale = FALSE)

In [ ]:
fviz_pca_ind(
res_pca,
geom = "point",
habillage = df_filtrado$clase_ternaria,
addEllipses = TRUE,
title = "PCA (2 componentes) Coloreado por clase"
)

UNMAP

In [ ]:
set.seed(123)
umap_emb <- umap(df_scaled, n_neighbors = 30, min_dist = 0.3)

df_umap <- as.data.frame(umap_emb)
df_umap$clase <- df_filtrado$clase

Graficamos UMAP por clase

In [ ]:
ggplot(df_umap, aes(V1, V2, color = clase)) +
geom_point(alpha = 0.7, size = 1.5) +
theme_minimal() +
labs(title = "UMAP – coloreado por clase")

DBSCAN sobre UMAP

In [ ]:
db <- dbscan(df_umap[,1:2], eps = 0.8, minPts = 20)

df_umap$cluster <- factor(db$cluster)

Graficar clusters DBSCAN

In [ ]:
ggplot(df_umap, aes(V1, V2, color = cluster)) +
geom_point(alpha = 0.7, size = 1.7) +
theme_minimal() +
labs(title = "DBSCAN sobre UMAP")

Cruzar cluster ↔ clase

In [ ]:
table(df_umap$cluster, df_umap$clase)

Visualizar densidades por cluster

In [ ]:
ggplot(df_umap, aes(V1, fill = cluster)) +
geom_density(alpha = 0.4) +
theme_minimal() +
labs(title = "Densidad de UMAP V1 por cluster")